In [254]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re
from spacy_langdetect import LanguageDetector
import spacy
from langdetect import detect_langs


In [1]:
from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Scraping Demand

In [2]:
# Load data
pages = pd.read_csv("data/safe_obs_dataset.csv", sep=";", index_col = "Unnamed: 0")

## Collect available timestamps

In [3]:
wayback_snapshots = ["20201207","20201215", "20201219", "20201224",  "20220101", "20220110", "20220122", "20220123", "20220126", "20220128", "20220204","20220211", "20220223", "20220224","20220301","20220304","20220307","20220308", "20220330", "20220402","20220403","202204415","20220417","20220424","20220429","20220509", "20220510","20220518","20220519","20220526","20220530","20210105", "20210110","20210117", "20210118", "20210123","20210124","20210125","20210126","20210131","20210201", "20210224", "20210307","20210314","20210317","20210319","20210322","20210323","20210324","20210325","20210326","20210330","20210408","20210411","20210421","20210429","20210601","20210608","20210612","20210614","20210801","20210806","20210811","20210812","20210816","20210825","20210902","20210904","20210926","20210929","20211004","20211005","20211006","20211008","20211013","20211015","20211015","20211025","20211027","20211029","20211031","20211105","20211115","20211117","20211209","20211213","20211216","20211217"]

In [8]:
pages["googleplay_shortlink"] = pages["googleplay_link"].str.replace("https://","").str.strip("/")

In [10]:
dict_gp = pages[pages["googleplay_shortlink"].isnull()==False][["url", "googleplay_shortlink"]].to_dict(orient="records")

In [11]:
dict_gp

[{'url': '/t/app-6-0-compass-of-chance.4440211/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=com.baiumka.compass_of_chance'},
 {'url': '/t/app-6-0-netguard-no-root-firewall.3233012/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=eu.faircode.netguard'},
 {'url': '/t/app-11-dual-camera-video-recorder.4447581/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=com.helge.dualcameravideorecorder'},
 {'url': '/t/app-4-0-true-phone-best-dialer-contact-manager-replacement.2934816/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=com.hb.dialer.free&referrer=utm_source%3Dforums%26utm_medium%3Dxda'},
 {'url': '/t/app-root-4-0-battery-charge-limit-v1-1-1.3557002/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=rs.pedjaapps.alogcatroot.app'},
 {'url': '/t/app-free-4-1-warm-up-exercises-and-training.4447459/',
  'googleplay_shortlink': 'play.google.com/store/apps/details?id=com.morningexercises.dzmitry'},


In [99]:
empty_att = []
try_again = []

In [ ]:
#snapshots = []
#counter=0

for date in wayback_snapshots:
    for gp_link in dict_gp:
        if gp_link["googleplay_shortlink"] not in empty_att:
            gp_page ={}
            
            gp_page["gp_url"] = gp_link["googleplay_shortlink"] 
            gp_page["try_url"] = date
            if gp_page.copy() not in finished_dict:
                #print("new")
                url = "http://archive.org/wayback/available?url=" + gp_link["googleplay_shortlink"] + '&timestamp=' + date
                r = requests.get(url)
                if r:
                    if len(r.json()["archived_snapshots"])>0:
                        if r.json()["archived_snapshots"]["closest"]["status"] == "200":
                            gp_page["gp_url"] = gp_link["googleplay_shortlink"] 
                            gp_page["timestamp"] = r.json()["archived_snapshots"]["closest"]["timestamp"]
                            gp_page["wb_url"] = r.json()["archived_snapshots"]["closest"]["url"]
                            check_d = {k: v for k, v in gp_page.copy().items() if k != 'try_url'}
                            check_list = [{k: v for k, v in d.items() if k != 'try_url'} for d in snapshots]
                            if check_d.copy() not in check_list:
                                snapshots.append(gp_page.copy())
                    else:
                        empty_att.append(gp_page["gp_url"])
                else:
                    try_again.append(gp_page)
                
    counter += 1
    print(counter)

24


In [124]:
finished = pd.DataFrame(snapshots)
finished

,gp_url,timestamp,wb_url,try_url
0,play.google.com/store/apps/details?id=com.baiu...,20220426080658,http://web.archive.org/web/20220426080658/http...,NaN
1,play.google.com/store/apps/details?id=eu.fairc...,20201128175644,http://web.archive.org/web/20201128175644/http...,NaN
2,play.google.com/store/apps/details?id=com.helg...,20220519073537,http://web.archive.org/web/20220519073537/http...,NaN
3,play.google.com/store/apps/details?id=com.hb.d...,20201204194737,http://web.archive.org/web/20201204194737/http...,NaN
4,play.google.com/store/apps/details?id=rs.pedja...,20201111200810,http://web.archive.org/web/20201111200810/http...,NaN
...,...,...,...,...
7175,play.google.com/store/apps/details?id=cdev.life,20220404215837,http://web.archive.org/web/20220404215837/http...,20220403
7176,play.google.com/store/apps/details?id=com.andr...,20220404011200,http://web.archive.org/web/20220404011200/http...,20220403
7177,play.google.com/store/apps/details?id=com.what...,20220403175737,http://web.archive.org/web/20220403175737/http...,20220403
7178,play.google.com/store/apps/details?id=com.skyp...,20220405031549,http://web.archive.org/web/20220405031549/http...,20220403


In [125]:
finished.loc[finished["try_url"].isnull() == True, "try_url"] = "20201207"

In [126]:
finished

,gp_url,timestamp,wb_url,try_url
0,play.google.com/store/apps/details?id=com.baiu...,20220426080658,http://web.archive.org/web/20220426080658/http...,20201207
1,play.google.com/store/apps/details?id=eu.fairc...,20201128175644,http://web.archive.org/web/20201128175644/http...,20201207
2,play.google.com/store/apps/details?id=com.helg...,20220519073537,http://web.archive.org/web/20220519073537/http...,20201207
3,play.google.com/store/apps/details?id=com.hb.d...,20201204194737,http://web.archive.org/web/20201204194737/http...,20201207
4,play.google.com/store/apps/details?id=rs.pedja...,20201111200810,http://web.archive.org/web/20201111200810/http...,20201207
...,...,...,...,...
7175,play.google.com/store/apps/details?id=cdev.life,20220404215837,http://web.archive.org/web/20220404215837/http...,20220403
7176,play.google.com/store/apps/details?id=com.andr...,20220404011200,http://web.archive.org/web/20220404011200/http...,20220403
7177,play.google.com/store/apps/details?id=com.what...,20220403175737,http://web.archive.org/web/20220403175737/http...,20220403
7178,play.google.com/store/apps/details?id=com.skyp...,20220405031549,http://web.archive.org/web/20220405031549/http...,20220403


In [57]:
finished_dict = finished[["gp_url", "try_url"]].to_dict(orient="records")

In [79]:
old = pd.DataFrame(dict_gp)

In [90]:
old

,url,googleplay_shortlink
0,/t/app-6-0-compass-of-chance.4440211/,play.google.com/store/apps/details?id=com.baiu...
1,/t/app-6-0-netguard-no-root-firewall.3233012/,play.google.com/store/apps/details?id=eu.fairc...
2,/t/app-11-dual-camera-video-recorder.4447581/,play.google.com/store/apps/details?id=com.helg...
3,/t/app-4-0-true-phone-best-dialer-contact-mana...,play.google.com/store/apps/details?id=com.hb.d...
4,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,play.google.com/store/apps/details?id=rs.pedja...
...,...,...
5832,/t/app-sequitimer.1498223/,play.google.com/store/apps/details?id=com.plum...
5833,/t/app-free-fu-ing-lousy-advice.1478522/,play.google.com/store/apps/details?id=ru.andre...
5834,/t/app-fastmeteo.1337572/,play.google.com/store/apps/details?id=com.app....
5835,/t/app-advanced-app-locker-free.1103940/,play.google.com/store/apps/details?id=com.antt...


In [93]:
fin = pd.merge(old, finished[finished["try_url"]=="20201207"], left_on = "googleplay_shortlink", right_on = "gp_url", how = "outer")

In [96]:
# I assume that if there was no closest snapshot, then there are no snapshots at all for this page!
empty = list(fin[fin["wb_url"].isnull()]["googleplay_shortlink"])

In [127]:
finished = finished.drop_duplicates(subset = ["gp_url", "timestamp"])

In [129]:
finished.to_csv("data/av_gplinks.csv")

In [130]:
finished

,gp_url,timestamp,wb_url,try_url
0,play.google.com/store/apps/details?id=com.baiu...,20220426080658,http://web.archive.org/web/20220426080658/http...,20201207
1,play.google.com/store/apps/details?id=eu.fairc...,20201128175644,http://web.archive.org/web/20201128175644/http...,20201207
2,play.google.com/store/apps/details?id=com.helg...,20220519073537,http://web.archive.org/web/20220519073537/http...,20201207
3,play.google.com/store/apps/details?id=com.hb.d...,20201204194737,http://web.archive.org/web/20201204194737/http...,20201207
4,play.google.com/store/apps/details?id=rs.pedja...,20201111200810,http://web.archive.org/web/20201111200810/http...,20201207
...,...,...,...,...
7175,play.google.com/store/apps/details?id=cdev.life,20220404215837,http://web.archive.org/web/20220404215837/http...,20220403
7176,play.google.com/store/apps/details?id=com.andr...,20220404011200,http://web.archive.org/web/20220404011200/http...,20220403
7177,play.google.com/store/apps/details?id=com.what...,20220403175737,http://web.archive.org/web/20220403175737/http...,20220403
7178,play.google.com/store/apps/details?id=com.skyp...,20220405031549,http://web.archive.org/web/20220405031549/http...,20220403


## Scrape available pages

In [289]:
# Batch 0:
#gp_df = pd.read_csv("data/av_gplinks.csv", index_col="Unnamed: 0")
#Batch 1&2:
gp_df_1 = pd.read_csv("data/av_gplinks_batch1.csv", sep = ";", index_col = "Unnamed: 0")
gp_df_2 = pd.read_csv("data/av_gplinks_batch2.csv", sep = ";", index_col="Unnamed: 0")

In [305]:
gp_df_1

,gp_url,timestamp,wb_url,try_url
0,play.google.com/store/apps/details?id=com.baiu...,20220426080658,http://web.archive.org/web/20220426080658/http...,20201207
1,play.google.com/store/apps/details?id=eu.fairc...,20201128175644,http://web.archive.org/web/20201128175644/http...,20201207
2,play.google.com/store/apps/details?id=com.helg...,20220519073537,http://web.archive.org/web/20220519073537/http...,20201207
3,play.google.com/store/apps/details?id=com.hb.d...,20201204194737,http://web.archive.org/web/20201204194737/http...,20201207
4,play.google.com/store/apps/details?id=rs.pedja...,20201111200810,http://web.archive.org/web/20201111200810/http...,20201207
...,...,...,...,...
4875,play.google.com/store/apps/details?id=com.andr...,20210314232240,http://web.archive.org/web/20210314232240/http...,20210314
4876,play.google.com/store/apps/details?id=com.them...,20210502085151,http://web.archive.org/web/20210502085151/http...,20210314
4877,play.google.com/store/apps/details?id=org.thou...,20210314233052,http://web.archive.org/web/20210314233052/http...,20210314
4878,play.google.com/store/apps/details?id=com.vsco...,20210318213639,http://web.archive.org/web/20210318213639/http...,20210314


In [302]:
gp_df_1["timestamp"] = gp_df_1["timestamp"].astype(str)
gp_df_2["timestamp"] = gp_df_2["timestamp"].astype(str)

In [303]:
gp_new = pd.concat([gp_df_1, gp_df_2]).drop_duplicates(subset=["gp_url", "timestamp"]).reset_index(drop=True)

In [312]:
gp_new

,gp_url,timestamp,wb_url,try_url
0,play.google.com/store/apps/details?id=com.baiu...,20220426080658,http://web.archive.org/web/20220426080658/http...,20201207
1,play.google.com/store/apps/details?id=eu.fairc...,20201128175644,http://web.archive.org/web/20201128175644/http...,20201207
2,play.google.com/store/apps/details?id=com.helg...,20220519073537,http://web.archive.org/web/20220519073537/http...,20201207
3,play.google.com/store/apps/details?id=com.hb.d...,20201204194737,http://web.archive.org/web/20201204194737/http...,20201207
4,play.google.com/store/apps/details?id=rs.pedja...,20201111200810,http://web.archive.org/web/20201111200810/http...,20201207
...,...,...,...,...
10867,play.google.com/store/apps/details?id=com.goog...,20211216110737,http://web.archive.org/web/20211216110737/http...,20211216
10868,play.google.com/store/apps/details?id=it.mirko...,20211220222811,http://web.archive.org/web/20211220222811/http...,20211217
10869,play.google.com/store/apps/details?id=com.andr...,20211218001854,http://web.archive.org/web/20211218001854/http...,20211217
10870,play.google.com/store/apps/details?id=org.thou...,20211217233132,http://web.archive.org/web/20211217233132/http...,20211217


In [315]:
# When scraping in batches, use: 

# df = gp_new.merge(gp_df, on=gp_new.columns.to_list(), 
#                    how='left', indicator=True).drop_duplicates(subset=["gp_url", "timestamp"])
# df_next = df.loc[df._merge=='left_only',df.columns!='_merge'].reset_index(drop=True)
# gp_dict = df_next[["gp_url", "wb_url", "timestamp"]].to_dict(orient="records")


In [320]:
counter = 0

In [359]:
for gp_page in gp_dict:
    url = gp_page["wb_url"]
    r = requests.get(url)
    time.sleep(5)
    if r:
        soup = BeautifulSoup(r.text)
        if soup.find('h1') is not None:
            gp_page["app_name"] = soup.find('h1').get_text()
        elif soup.find(class_="document-title") is not None:
            gp_page["app_name"] = soup.find(class_="document-title").get_text().strip()
        else:
            gp_page["app_name"] = soup.find(class_="document-title").get_text().strip()
        gp_page["ads"] = re.search("Ads", soup.text) is not None
        gp_page["paid_features"] = re.search("In-app Products", soup.text) is not None
        
        if re.search("(?<=Installs)\d[\d,]*\+", soup.text) is not None:
            gp_page["demand"] = re.search("(?<=Installs)\d[\d,]*\+", soup.text).group(0)
            gp_page["demand_flag"] = False
        elif re.search("(?<=Installs)\d[0,]* \- \d[0,]*", soup.text) is not None: 
            gp_page["demand"]= re.search("(?<=Installs)\d[0,]* \- \d[0,]*", soup.text).group(0)
            gp_page["demand_flag"] = False
        
        elif re.search("\d[\d,]*\+", soup.text) is not None:
            gp_page["demand"] = re.search("\d[\d,]*\+", soup.text).group(0)
            
        gp_page["paid_new"] = len(re.findall('aria-label=".+ Buy">', r.text)) > 0
        gp_page["language_flag"]=len(re.findall("[\[\]^en0-9\.:]", str(detect_langs(soup.text))))>0
    counter += 1
    if counter % 100 == 0:
        print(counter)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
finished = pd.DataFrame(gp_dict)

In [232]:
#fin.to_csv("data/gp_timestamps_batch0.csv", sep=";")
finished.to_csv("data/gp_timestamps.csv", sep=";")